In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('chat1.txt',header=None,error_bad_lines=False,encoding='utf8')

C:\Users\lavanya\anaconda3\envs\aiml\lib\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 181: expected 2 fields, saw 3\nSkipping line 183: expected 2 fields, saw 10\nSkipping line 222: expected 2 fields, saw 3\n'


In [3]:
df

,0,1
0,31/05/2024,1:20 pm - Leena: IMG-20240531-WA0025.jpg (fil...
1,31/05/2024,1:21 pm - Leena: IMG-20240531-WA0024.jpg (fil...
2,31/05/2024,1:21 pm - Leena: IMG-20240531-WA0027.jpg (fil...
3,31/05/2024,1:22 pm - Lavanya: Final answer??
4,31/05/2024,1:22 pm - Lavanya: H(z) vendama??
...,...,...
208,12/06/2024,4:13 pm - Leena: 😳
209,12/06/2024,4:13 pm - Leena: Idhenna pudhusa iruku
210,12/06/2024,4:13 pm - Lavanya: Ithukku thaan avanga reply...
211,12/06/2024,4:13 pm - Leena: Apdiya


In [4]:
df=df.drop(0)
df=df.drop(1)
df=df.drop(2)
df.columns=['Date','Chat']
Message=df["Chat"].str.split("-",n=1,expand=True)
df["Time"]=Message[0]
Message1=Message[1].str.split(":",n=1,expand=True)
df["Name"]=Message1[0]
df["Chat"]=Message1[1]
df=df[["Date","Time","Name","Chat"]]
df

,Date,Time,Name,Chat
3,31/05/2024,1:22 pm,Lavanya,Final answer??
4,31/05/2024,1:22 pm,Lavanya,H(z) vendama??
5,31/05/2024,1:22 pm,Leena,Venum di
6,31/05/2024,1:23 pm,Leena,Potutu anupren wait pannu
7,31/05/2024,1:23 pm,Lavanya,Mmm
...,...,...,...,...
208,12/06/2024,4:13 pm,Leena,😳
209,12/06/2024,4:13 pm,Leena,Idhenna pudhusa iruku
210,12/06/2024,4:13 pm,Lavanya,Ithukku thaan avanga reply pannanga
211,12/06/2024,4:13 pm,Leena,Apdiya


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 3 to 212
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    210 non-null    object
 1   Time    164 non-null    object
 2   Name    163 non-null    object
 3   Chat    163 non-null    object
dtypes: object(4)
memory usage: 8.2+ KB


In [6]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 3 to 212
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    163 non-null    object
 1   Time    163 non-null    object
 2   Name    163 non-null    object
 3   Chat    163 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Ensure the VADER lexicon is downloaded
nltk.download('vader_lexicon')

def sentimentalAnalysis(data, columnname):
    """Perform sentiment analysis on a specific column in the dataframe using VADER."""
    
    # Initialize the sentiment intensity analyzer
    sid = SentimentIntensityAnalyzer()
    
    # Drop any rows with null or empty values in the specified column
    data = data.dropna(subset=[columnname])
    
    # Apply sentiment analysis
    data['scores'] = data[columnname].apply(lambda commentText: sid.polarity_scores(commentText))
    data['compound'] = data['scores'].apply(lambda score_dict: score_dict['compound'])
    data['Negative'] = data['scores'].apply(lambda score_dict: score_dict['neg'])
    data['Positive'] = data['scores'].apply(lambda score_dict: score_dict['pos'])
    data['Neutral'] = data['scores'].apply(lambda score_dict: score_dict['neu'])
    
    # Create a column for final positive or negative sentiment based on compound score
    data['comp_score'] = data['compound'].apply(lambda c: 'pos' if c >= 0 else 'neg')
    
    # Clear any previous plots
    plt.clf()
    
    # Count the number of positive and negative sentiments
    postneg = pd.DataFrame(data['comp_score'].value_counts())
    
    return postneg, data

In [ ]:
pos,data_senti=sentimentalAnalysis(df,columnname='Chat')

In [ ]:
data_senti

In [ ]:
pos

In [ ]:
df = pd.concat([ data_senti], axis=1)
df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#create vector
tfidf=TfidfVectorizer(max_df=0.95,min_df=2,stop_words='english')
dtm=tfidf.fit_transform(df["Chat"])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,random_state=42)
lda_model.fit(dtm)

In [ ]:
for index,topic in enumerate(lda_model.components_):
    results=([tfidf.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print(results)

In [ ]:
topic_results=lda_model.transform(dtm)
df["Topic"]=topic_results.argmax(axis=1)

In [ ]:
df["Topic"].value_counts()

In [ ]:
df

In [ ]:
df

# Talkative vs Less Talkative

In [ ]:
# Group by 'Name' to count messages sent by each person
message_counts = df.groupby('Name').size().reset_index(name='MessageCount')

# Identify talkative vs. less talkative based on message counts
talkative_threshold = message_counts['MessageCount'].quantile(0.75)
message_counts['Talkative'] = message_counts['MessageCount'].apply(lambda count: 'Talkative' if count >= talkative_threshold else 'Less Talkative')

# Display message counts and talkative classification
print("\nMessage Counts per Person:")
print(message_counts)

# Visualize using seaborn or matplotlib as needed
sns.barplot(x='Name', y='MessageCount', data=message_counts, hue='Talkative')
plt.xticks(rotation=45)
plt.title('Message Counts and Talkative Classification')
plt.show()


# Media Count Sent by Each Person

In [ ]:
# Count 'Media' occurrences in the 'Chat' column
df['MediaCount'] = df['Chat'].str.count('<Media omitted>')

# Group by 'Name' to sum media counts
media_counts = df.groupby('Name')['MediaCount'].sum().reset_index()

# Display media counts per person
print("\nMedia Counts per Person:")
print(media_counts)


# Missed Call

In [ ]:
# Count occurrences of 'Missed call'
df['MissedCallCount'] = df['Chat'].str.count('Missed call')

# Sum missed call counts per person
missed_calls = df.groupby('Name')['MissedCallCount'].sum().reset_index()

# Display missed call counts per person
print("\nMissed Call Counts per Person:")
print(missed_calls)


# Missed Video

In [ ]:
# Count occurrences of 'Missed call'
df['MissedvideoCount'] = df['Chat'].str.count('Missed video')

# Sum missed call counts per person
missed_video = df.groupby('Name')['MissedvideoCount'].sum().reset_index()

# Display missed call counts per person
print("\nMissed video Counts per Person:")
print(missed_video)


# deleted messages

In [ ]:
# Count occurrences of 'Missed call'
df['deletedmessageCount'] = df['Chat'].str.count('deleted message')

# Sum missed call counts per person
deletedmessage = df.groupby('Name')['deletedmessageCount'].sum().reset_index()

# Display missed call counts per person
print("\ndeletedmessage Counts per Person:")
print(deletedmessage)


# Flirt and non-flirt

In [ ]:
flirt_keywords = ['flirt', 'date', 'cute', 'handsome', 'beautiful', 'attractive', 'charming', 
                  'sweet', 'gorgeous', 'lovely', 'heart', 'kiss', 'hug', 'romantic', 'babe', 
                  'darling', 'adorable', 'hot', 'wink']

In [ ]:
def assign_label(chat_message):
    for word in flirt_keywords:
        if word in chat_message.lower():
            return 'flirt'
    return 'non-flirt'


In [ ]:
df['Label'] = df['Chat'].apply(assign_label)

In [ ]:
df = df[['Name','Chat', 'Label']]
df=pd.DataFrame(df)
df